## Pipeline Tutorial with HeteroSecureBoost

### install

`Pipeline` is distributed along with [fate_client](https://pypi.org/project/fate-client/).

```bash
pip install fate_client
```

To use Pipeline, we need to first specify which `FATE Flow Service` to connect to. Once `fate_client` installed, one can find an cmd enterpoint name `pipeline`:

In [12]:
import sys
print(sys.version)

3.8.13 (default, Oct 21 2022, 23:50:54) 
[GCC 11.2.0]


In [13]:
!python --version

Python 3.8.13


In [14]:
!pipeline --help

Usage: pipeline [OPTIONS] COMMAND [ARGS]...

Options:
  --help  Show this message and exit.

Commands:
  config  pipeline config tool
  init    - DESCRIPTION: Pipeline Config Command.


Assume we have a `FATE Flow Service` in 127.0.0.1:9380(defaults in standalone), then exec

In [22]:
!pipeline init --ip 0.0.0.0 --port 9380

Pipeline configuration succeeded.


### Hetero SecureBoost Example

 Before start a modeling task, data to be used should be uploaded. Please refer to this [guide](https://github.com/FederatedAI/FATE/blob/master/doc/tutorial/pipeline/pipeline_tutorial_upload.ipynb).

The `pipeline` package provides components to compose a `FATE pipeline`.

In [23]:
from pipeline.backend.pipeline import PipeLine
from pipeline.component import Reader, DataTransform, Intersection, HeteroSecureBoost, Evaluation
from pipeline.interface import Data

Make a `pipeline` instance:

    - initiator: 
        * role: guest
        * party: 9999
    - roles:
        * guest: 9999
        * host: 10000
    

In [24]:
pipeline = PipeLine() \
        .set_initiator(role='guest', party_id=9999) \
        .set_roles(guest=9999, host=10000)

Now we need to upload our data.

In [25]:
import os

data_base = os.path.join(os.getcwd(), "../data/")
partition = 4
guest = 9999

dense_data = {"name": "breast_hetero_guest", "namespace": f"experiment"}
tag_data = {"name": "breast_hetero_host", "namespace": f"experiment"}

In [26]:
#pipeline_upload = PipeLine().set_initiator(role="guest", party_id=guest).set_roles(guest=guest)
pipeline_upload = PipeLine().set_initiator(role='guest', party_id=9999).set_roles(guest=9999)

In [27]:
pipeline_upload.add_upload_data(file=os.path.join(data_base, "breast_hetero_guest.csv"),
                                table_name=dense_data["name"],             # table name
                                namespace=dense_data["namespace"],         # namespace
                                head=1, partition=partition)               # data info


pipeline_upload.add_upload_data(file=os.path.join(data_base, "breast_hetero_host.csv"),
                                table_name=tag_data["name"],
                                namespace=tag_data["namespace"],
                                head=1, partition=partition)

In [29]:
pipeline_upload.upload(drop=1)

 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||56.00%

2023-05-25 15:40:19.062 | ERROR    | __main__:<module>:1 - An error has been caught in function '<module>', process 'MainProcess' (21311), thread 'MainThread' (140069943566976):
Traceback (most recent call last):

  File "/home/paul/.conda/envs/fate/lib/python3.8/site-packages/pipeline/utils/invoker/job_submitter.py", line 61, in upload_data
    raise ValueError

ValueError


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "/home/paul/.conda/envs/fate/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
           │         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
           │         └ <code object <module> at 0x7f6492e57a80, file "/home/paul/.conda/envs/fate/lib/python3.8/site-packages/ipykernel_launcher.py"...
           └ <function _run_code at 0x7f6492e68160>
  File "/hom

ValueError: job submit failed, err msg: {'retcode': 100, 'retmsg': "('Connection aborted.', BadStatusLine('\\x00\\x00\\x18\\x04\\x00\\x00\\x00\\x00\\x00\\x00\\x04\\x00?ÿÿ\\x00\\x05\\x00?ÿÿ\\x00\\x06\\x00\\x00 \\x00þ\\x03\\x00\\x00\\x00\\x01\\x00\\x00\\x04\\x08\\x00\\x00\\x00\\x00\\x00\\x00?\\x00\\x00'))", 'traceback': ['Traceback (most recent call last):\n', '  File "/home/paul/.conda/envs/fate/lib/python3.8/site-packages/urllib3/connectionpool.py", line 714, in urlopen\n    httplib_response = self._make_request(\n', '  File "/home/paul/.conda/envs/fate/lib/python3.8/site-packages/urllib3/connectionpool.py", line 466, in _make_request\n    six.raise_from(e, None)\n', '  File "<string>", line 3, in raise_from\n', '  File "/home/paul/.conda/envs/fate/lib/python3.8/site-packages/urllib3/connectionpool.py", line 461, in _make_request\n    httplib_response = conn.getresponse()\n', '  File "/home/paul/.conda/envs/fate/lib/python3.8/http/client.py", line 1348, in getresponse\n    response.beg

Define a `Reader` to load data

In [30]:
reader_0 = Reader(name="reader_0")
# set guest parameter
reader_0.get_party_instance(role='guest', party_id=9999).component_param(
    table={"name": "breast_hetero_guest", "namespace": "experiment"})
# set host parameter
reader_0.get_party_instance(role='host', party_id=10000).component_param(
    table={"name": "breast_hetero_host", "namespace": "experiment"})

Add a `DataTransform` component to parse raw data into Data Instance

In [31]:
data_transform_0 = DataTransform(name="data_transform_0")
# set guest parameter
data_transform_0.get_party_instance(role='guest', party_id=9999).component_param(
    with_label=True)
data_transform_0.get_party_instance(role='host', party_id=[10000]).component_param(
    with_label=False)

Add a `Intersection` component to perform PSI for hetero-scenario

In [32]:
intersect_0 = Intersection(name="intersect_0")

Now, we define the `HeteroSecureBoost` component. The following parameters will be set for all parties involved.

In [33]:
hetero_secureboost_0 = HeteroSecureBoost(name="hetero_secureboost_0",
                                         num_trees=5,
                                         bin_num=16,
                                         task_type="classification",
                                         objective_param={"objective": "cross_entropy"},
                                         encrypt_param={"method": "paillier"},
                                         tree_param={"max_depth": 3})


To show the evaluation result, an "Evaluation" component is needed.

In [34]:
evaluation_0 = Evaluation(name="evaluation_0", eval_type="binary")

Add components to pipeline, in order of execution:

    - data_transform_0 comsume reader_0's output data
    - intersect_0 comsume data_transform_0's output data
    - hetero_secureboost_0 consume intersect_0's output data
    - evaluation_0 consume hetero_secureboost_0's prediciton result on training data

Then compile our pipeline to make it ready for submission.

In [35]:
pipeline.add_component(reader_0)
pipeline.add_component(data_transform_0, data=Data(data=reader_0.output.data))
pipeline.add_component(intersect_0, data=Data(data=data_transform_0.output.data))
pipeline.add_component(hetero_secureboost_0, data=Data(train_data=intersect_0.output.data))
pipeline.add_component(evaluation_0, data=Data(data=hetero_secureboost_0.output.data))
pipeline.compile();


Now, submit(fit) our pipeline:

In [36]:
pipeline.fit()


2023-05-25 15:51:14.561 | ERROR    | __main__:<module>:1 - An error has been caught in function '<module>', process 'MainProcess' (21311), thread 'MainThread' (140069943566976):
Traceback (most recent call last):

  File "/home/paul/.conda/envs/fate/lib/python3.8/site-packages/pipeline/utils/invoker/job_submitter.py", line 46, in submit_job
    raise ValueError(f"retcode err, callback result is {result}")

ValueError: retcode err, callback result is {'retcode': 100, 'retmsg': "('Connection aborted.', BadStatusLine('\\x00\\x00\\x18\\x04\\x00\\x00\\x00\\x00\\x00\\x00\\x04\\x00?ÿÿ\\x00\\x05\\x00?ÿÿ\\x00\\x06\\x00\\x00 \\x00þ\\x03\\x00\\x00\\x00\\x01\\x00\\x00\\x04\\x08\\x00\\x00\\x00\\x00\\x00\\x00?\\x00\\x00'))", 'traceback': ['Traceback (most recent call last):\n', '  File "/home/paul/.conda/envs/fate/lib/python3.8/site-packages/urllib3/connectionpool.py", line 714, in urlopen\n    httplib_response = self._make_request(\n', '  File "/home/paul/.conda/envs/fate/lib/python3.8/site-package

ValueError: job submit failed, err msg: {'retcode': 100, 'retmsg': "('Connection aborted.', BadStatusLine('\\x00\\x00\\x18\\x04\\x00\\x00\\x00\\x00\\x00\\x00\\x04\\x00?ÿÿ\\x00\\x05\\x00?ÿÿ\\x00\\x06\\x00\\x00 \\x00þ\\x03\\x00\\x00\\x00\\x01\\x00\\x00\\x04\\x08\\x00\\x00\\x00\\x00\\x00\\x00?\\x00\\x00'))", 'traceback': ['Traceback (most recent call last):\n', '  File "/home/paul/.conda/envs/fate/lib/python3.8/site-packages/urllib3/connectionpool.py", line 714, in urlopen\n    httplib_response = self._make_request(\n', '  File "/home/paul/.conda/envs/fate/lib/python3.8/site-packages/urllib3/connectionpool.py", line 466, in _make_request\n    six.raise_from(e, None)\n', '  File "<string>", line 3, in raise_from\n', '  File "/home/paul/.conda/envs/fate/lib/python3.8/site-packages/urllib3/connectionpool.py", line 461, in _make_request\n    httplib_response = conn.getresponse()\n', '  File "/home/paul/.conda/envs/fate/lib/python3.8/http/client.py", line 1348, in getresponse\n    response.beg

Once training is done, trained model may be used for prediction. Optionally, save the trained pipeline for future use.

In [12]:
pipeline.dump("pipeline_saved.pkl");

First, deploy needed components from train pipeline

In [13]:
pipeline = PipeLine.load_model_from_file('pipeline_saved.pkl')
pipeline.deploy_component([pipeline.data_transform_0, pipeline.intersect_0, pipeline.hetero_secureboost_0]);

Define new `Reader` components for reading prediction data

In [14]:
reader_1 = Reader(name="reader_1")
reader_1.get_party_instance(role="guest", party_id=9999).component_param(table={"name": "breast_hetero_guest", "namespace": "experiment"})
reader_1.get_party_instance(role="host", party_id=10000).component_param(table={"name": "breast_hetero_host", "namespace": "experiment"})

Optionally, define new `Evaluation` component.

In [15]:
evaluation_0 = Evaluation(name="evaluation_0", eval_type="binary")

Add components to predict pipeline in order of execution:

In [16]:
predict_pipeline = PipeLine()
predict_pipeline.add_component(reader_1)\
                .add_component(pipeline, 
                               data=Data(predict_input={pipeline.data_transform_0.input.data: reader_1.output.data}))\
                .add_component(evaluation_0, data=Data(data=pipeline.hetero_secureboost_0.output.data));


Then, run prediction job

In [17]:
predict_pipeline.predict()

2021-12-31 03:25:35.541 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:123 - Job id is 202112310325328444510

2021-12-31 03:25:47.384 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:11
m2021-12-31 03:25:47.903 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:173 - 
2021-12-31 03:25:52.078 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component reader_1, time elapse: 0:00:16
m2021-12-31 03:25:54.161 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:173 - 
2021-12-31 03:25:58.545 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component data_transform_0, time elapse: 0:00:23
m2021-12-31 03:26:01.167 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:173 - 
2021-12-31 03:26:07.502 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component in

For more demo on using pipeline to submit jobs, please refer to [pipeline demos](https://github.com/FederatedAI/FATE/tree/master/examples/pipeline/demo)